In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_8_day()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_bnd3,wp_bnd7,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.032962,0.136763,0.396594,0.353988,0.641934,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.032200,0.131800,0.407211,0.376832,0.654545,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.031425,0.128250,0.416158,0.394189,0.664269,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.030650,0.124700,0.425106,0.411547,0.673993,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.029875,0.121150,0.434053,0.428904,0.683716,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030 entries, 0 to 1029
Data columns (total 28 columns):
PET             1030 non-null float64
VPD             1030 non-null float64
air_temp        1030 non-null float64
doy             1030 non-null int64
precip          1030 non-null float64
soil_temp       1030 non-null float64
sw_in           1030 non-null float64
wind_speed      1030 non-null float64
year            1030 non-null int64
wp_RNET         1030 non-null float64
wp_ch4_gf       1030 non-null float64
wp_co2_gf       1030 non-null float64
wp_er           1030 non-null float64
wp_gpp          1030 non-null float64
wp_h            1030 non-null float64
wp_le           1030 non-null float64
wp_bnd1         1030 non-null float64
wp_bnd2         1030 non-null float64
wp_bnd3         1030 non-null float64
wp_bnd7         1030 non-null float64
wp_evi          1030 non-null float64
wp_lswi2        1030 non-null float64
wp_ndvi         1030 non-null float64
wp_LST.day      1030 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_le"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1030, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.972242943017
CV Scores:  0.971482432189, 0.979074937486, 0.976979369682, 0.975024005679, 0.974901666434, 0.971012929208, 0.961235014157, 0.975572262557, 0.975287990391, 0.961858822385
OOB score: 0.973995956298
Feature Importances:
('wp_LST.night', 0.28042171711997765)
('air_temp', 0.20053670307748298)
('wp_LST.day', 0.1292270428672368)
('PET', 0.11381427311001392)
('VPD', 0.086300438360465859)
('sw_in', 0.058679652074425437)
('LW_IN.wp', 0.035518488663551742)
('wp_evi', 0.034197633775860513)
('wp_ndvi', 0.029347472351898007)
('wp_lswi2', 0.017655839741032939)
('wind_speed', 0.013636574185553639)
('precip', 0.00066416467250073941)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.972096806054
CV Scores:  0.971336314606, 0.975157254221, 0.973457442416, 0.97470020109, 0.974779756703, 0.973704118861, 0.960768365744, 0.978455874824, 0.975490508222, 0.963118223853
Feature Importances:
('wp_LST.night', 0.13932250951915914)
('wp_lswi2', 0.12384523435851255)
('wp_evi', 0.12257958014427059)
('wp_LST.day', 0.11366148475934384)
('sw_in', 0.087778538178524435)
('wp_ndvi', 0.081251791572711715)
('air_temp', 0.080214143362961232)
('PET', 0.076214907079000749)
('VPD', 0.05857619142701731)
('wind_speed', 0.056732933318252)
('LW_IN.wp', 0.051341983486041597)
('precip', 0.0084807027942050313)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.955818900176
CV Scores:  0.962257254755, 0.955321370516, 0.944693524046, 0.963532718723, 0.972001769287, 0.959859695339, 0.940180813751, 0.967520472224, 0.941092117128, 0.951729265989


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 58.83918
Step #501, epoch #50, avg. loss: 2.03326
Step #1001, epoch #100, avg. loss: 0.68148
Step #1501, epoch #150, avg. loss: 0.39972
Step #2001, epoch #200, avg. loss: 0.28596
Step #2501, epoch #250, avg. loss: 0.22820
Step #3001, epoch #300, avg. loss: 0.17349
Step #3501, epoch #350, avg. loss: 0.14920
Step #4001, epoch #400, avg. loss: 0.12463
Step #4501, epoch #450, avg. loss: 0.11161
Step #1, avg. loss: 57.67524
Step #501, epoch #50, avg. loss: 2.04614
Step #1001, epoch #100, avg. loss: 0.74851
Step #1501, epoch #150, avg. loss: 0.44488
Step #2001, epoch #200, avg. loss: 0.33645
Step #2501, epoch #250, avg. loss: 0.25960
Step #3001, epoch #300, avg. loss: 0.21962
Step #3501, epoch #350, avg. loss: 0.17604
Step #4001, epoch #400, avg. loss: 0.15961
Step #4501, epoch #450, avg. loss: 0.14966
Step #1, avg. loss: 53.25545
Step #501, epoch #50, avg. loss: 1.99268
Step #1001, epoch #100, avg. loss: 0.65608
Step #1501, epoc

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x115330488>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)